# BTC Bollinger Bands Strategy (Beginner-Friendly Quant Notebook)

This notebook shows a complete mini quant workflow:

- Download BTC-USD daily data
- Build Bollinger Bands features
- Create a simple trading strategy
- Backtest it with fees
- Evaluate performance with standard risk metrics


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

## 1) Parameters

In [ ]:
SYMBOL = "BTC-USD"
START = "2017-01-01"
END = None  # None = up to latest available
BB_WINDOW = 20
BB_STD = 2.0

FEE_BPS = 10  # 10 bps = 0.10% per trade (rough, adjust as you like)
INITIAL_CAPITAL = 1.0

## 2) Download data

In [ ]:
df = yf.download(SYMBOL, start=START, end=END, auto_adjust=True, progress=False)
df = df.rename(columns=str.lower)

if df.empty:
    raise ValueError("No data returned. Check symbol or your internet connection.")

df = df[["open", "high", "low", "close", "volume"]].copy()
df.index = pd.to_datetime(df.index)
df.head()

## 3) Build Bollinger Bands features

In [ ]:
close = df["close"]

ma = close.rolling(BB_WINDOW).mean()
sd = close.rolling(BB_WINDOW).std(ddof=0)

df["bb_mid"] = ma
df["bb_up"] = ma + BB_STD * sd
df["bb_dn"] = ma - BB_STD * sd

df["ret"] = close.pct_change()
df["logret"] = np.log(close).diff()

df.dropna(inplace=True)
df.tail()

### Quick visual check

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df.index, df["close"], label="Close")
plt.plot(df.index, df["bb_mid"], label="BB Mid")
plt.plot(df.index, df["bb_up"], label="BB Upper")
plt.plot(df.index, df["bb_dn"], label="BB Lower")
plt.title(f"{SYMBOL} with Bollinger Bands ({BB_WINDOW}, {BB_STD}σ)")
plt.legend()
plt.show()

## 4) Strategy idea (simple and common)

A classic mean-reversion approach:

- **Buy (go long)** when price crosses below the lower band
- **Exit to cash** when price crosses above the middle band

This keeps the logic simple and avoids shorting.


In [ ]:
df["long_entry"] = (df["close"].shift(1) >= df["bb_dn"].shift(1)) & (df["close"] < df["bb_dn"])
df["long_exit"] = (df["close"].shift(1) <= df["bb_mid"].shift(1)) & (df["close"] > df["bb_mid"])

pos = np.zeros(len(df), dtype=float)
in_pos = 0.0
for i, (entry, exit_) in enumerate(zip(df["long_entry"].values, df["long_exit"].values)):
    if in_pos == 0.0 and entry:
        in_pos = 1.0
    elif in_pos == 1.0 and exit_:
        in_pos = 0.0
    pos[i] = in_pos

df["pos"] = pos
df[["close", "bb_dn", "bb_mid", "pos"]].tail()

## 5) Backtest with simple fees

- Strategy return = position × daily return
- Fee is charged when position changes (enter or exit)


In [ ]:
fee = FEE_BPS / 10_000

df["pos_shift"] = df["pos"].shift(1).fillna(0.0)
df["turnover"] = (df["pos"] - df["pos_shift"]).abs()  # 1.0 when entering/exiting

df["strat_ret_gross"] = df["pos_shift"] * df["ret"]
df["strat_fee"] = df["turnover"] * fee
df["strat_ret_net"] = df["strat_ret_gross"] - df["strat_fee"]

df["bh_ret"] = df["ret"]

df["equity_strat"] = (1.0 + df["strat_ret_net"]).cumprod() * INITIAL_CAPITAL
df["equity_bh"] = (1.0 + df["bh_ret"]).cumprod() * INITIAL_CAPITAL

df[["strat_ret_net", "equity_strat", "equity_bh"]].tail()

## 6) Performance metrics

We compute standard quant metrics on daily returns:

- CAGR
- Volatility (annualized)
- Sharpe ratio
- Sortino ratio
- Max drawdown
- Calmar ratio


In [ ]:
TRADING_DAYS = 365  # crypto trades every day

def max_drawdown(equity: pd.Series) -> float:
    peak = equity.cummax()
    dd = equity / peak - 1.0
    return float(dd.min())

def perf_stats(daily_ret: pd.Series, equity: pd.Series, turnover: pd.Series | None = None) -> dict:
    daily_ret = daily_ret.dropna()
    if len(daily_ret) < 2:
        return {}

    total_years = (equity.index[-1] - equity.index[0]).days / 365.25
    cagr = (equity.iloc[-1] / equity.iloc[0]) ** (1 / total_years) - 1 if total_years > 0 else np.nan

    vol = daily_ret.std(ddof=0) * np.sqrt(TRADING_DAYS)
    mean = daily_ret.mean() * TRADING_DAYS
    sharpe = mean / vol if vol > 0 else np.nan

    downside = daily_ret[daily_ret < 0]
    downside_vol = downside.std(ddof=0) * np.sqrt(TRADING_DAYS)
    sortino = mean / downside_vol if downside_vol > 0 else np.nan

    mdd = max_drawdown(equity)
    calmar = cagr / abs(mdd) if mdd < 0 else np.nan

    win_rate = (daily_ret > 0).mean()

    trades = int(turnover.sum()) if turnover is not None else np.nan

    return {
        "CAGR": cagr,
        "Volatility": vol,
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Max Drawdown": mdd,
        "Calmar": calmar,
        "Win Rate": win_rate,
        "Final Equity": float(equity.iloc[-1]),
        "Trades (approx)": trades,
    }

stats_strat = perf_stats(df["strat_ret_net"], df["equity_strat"], df["turnover"])
stats_bh = perf_stats(df["bh_ret"], df["equity_bh"], None)

out = pd.DataFrame([stats_strat, stats_bh], index=["Bollinger Strategy (net)", "Buy & Hold"])
out

## 7) Visualize equity curves and drawdowns

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df.index, df["equity_strat"], label="Strategy (net)")
plt.plot(df.index, df["equity_bh"], label="Buy & Hold")
plt.title("Equity Curve")
plt.legend()
plt.show()

peak = df["equity_strat"].cummax()
dd = df["equity_strat"] / peak - 1.0

plt.figure(figsize=(12, 3))
plt.plot(df.index, dd)
plt.title("Strategy Drawdown")
plt.show()

## 8) Simple parameter sweep (optional)

Quickly check how the strategy behaves for different Bollinger windows.
This is *not* a full research pipeline, but it's a useful beginner sanity check.


In [ ]:
def run_strategy(window: int, std_mult: float) -> dict:
    close = df["close"]
    ma = close.rolling(window).mean()
    sd = close.rolling(window).std(ddof=0)

    tmp = df.copy()
    tmp["bb_mid"] = ma
    tmp["bb_dn"] = ma - std_mult * sd
    tmp = tmp.dropna().copy()

    entry = (tmp["close"].shift(1) >= tmp["bb_dn"].shift(1)) & (tmp["close"] < tmp["bb_dn"])
    exit_ = (tmp["close"].shift(1) <= tmp["bb_mid"].shift(1)) & (tmp["close"] > tmp["bb_mid"])

    pos = np.zeros(len(tmp), dtype=float)
    in_pos = 0.0
    for i, (e, x) in enumerate(zip(entry.values, exit_.values)):
        if in_pos == 0.0 and e:
            in_pos = 1.0
        elif in_pos == 1.0 and x:
            in_pos = 0.0
        pos[i] = in_pos

    tmp["pos"] = pos
    tmp["pos_shift"] = tmp["pos"].shift(1).fillna(0.0)
    tmp["turnover"] = (tmp["pos"] - tmp["pos_shift"]).abs()

    fee = FEE_BPS / 10_000
    tmp["strat_ret_net"] = tmp["pos_shift"] * tmp["ret"] - tmp["turnover"] * fee
    tmp["equity"] = (1.0 + tmp["strat_ret_net"]).cumprod()

    s = perf_stats(tmp["strat_ret_net"], tmp["equity"], tmp["turnover"])
    s["Window"] = window
    s["Std"] = std_mult
    return s

windows = [10, 20, 30, 40]
rows = [run_strategy(w, BB_STD) for w in windows]
pd.DataFrame(rows).set_index("Window")[["CAGR","Sharpe","Max Drawdown","Final Equity","Trades (approx)"]]